<a href="https://colab.research.google.com/github/tefarov/netology_pyda5/blob/master/homework_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import datetime as dt
import sys

# Задание 1.
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [0]:
STR_PTHFOLD = sys.path[0]
STR_PTHRATG = STR_PTHFOLD + r"\ratings.csv"

# импорт данных и превращаем timestamp  в читаемую datetime
rat_df = pd.read_csv(STR_PTHRATG)
rat_df['time'] = rat_df['timestamp'].apply(lambda x: dt.datetime.fromtimestamp(x))
rat_df.head()

In [0]:
# группируем рэйтинги по пользователю
usr_gp = rat_df.groupby('userId')
# создаём таблицу пользователей и отбираме тех,
# у которых количество оценок соответствует заданному порогу
usr_df = usr_gp.count()[['time']]
usr_df = usr_df[usr_df.time > 100]
usr_df.head()

In [0]:
min_df = usr_gp.min()[['time']] # фрейм с временем первой оценки
max_df = usr_gp.max()[['time']] # фрейм с врменеем последней оценки
# объединяем по индексу всё в один фрейм
usr_df = (usr_df
            .join(min_df,how='left',rsuffix='_min')
            .join(max_df,how='left',rsuffix='_max')
            )
usr_df.head()

In [0]:
# расчёт средней продолжительности жизни для каждого пользователя
usr_df['lifetime'] = (usr_df['time_max'] - usr_df['time_min'])
# расчёт средней продолжительности жизни по всем отобранным пользователям
lft = usr_df['lifetime'].mean()

print("Средняя продолжительность жизни пользователя: {0} дн. {1} ч. {2} мин. {3} сек.".format(lft.components.days,lft.components.hours,lft.components.minutes,lft.components.seconds))

# Задание 2
Дана статистика услуг перевозок клиентов компании по типам
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

Необходимо сформировать две таблицы:
 1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
 2. аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [0]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [0]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [0]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [0]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

In [0]:
dat = (auto
    .merge(rzd,how='outer',on='client_id')
    .merge(air,how='outer',on='client_id')
)
dat

In [0]:
dat = (client_base
        .merge(rzd ,how='left',on='client_id')
        .merge(air ,how='left',on='client_id')
        .merge(auto,how='left',on='client_id')
    )
dat

# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 
Как бы вы добавили эти сведения в таблицу визитов и покупок?

## Исходные данные
Допустим у нас есть следующие наборы данных :
<br>**Гео-координаты по времени**. Предположим, что приложение, просто время от времени передаёт координаты и эти данные не привязаны ни к входам, ни к покупкам. Данные хранятся в ISO-формате для простоты чтения, для повышения скорости необходимо хранить timestamp.

In [7]:
geo_df = [    
    ( 1, '2019-03-01T10:14:00',54.88, 42.93),
    ( 1, '2019-03-01T15:20:00',57.89, 48.04),
    ( 1, '2019-03-01T18:46:00',51.61, 44.52),
    ( 1, '2019-03-02T02:42:00',59.18, 45.20),
    ( 1, '2019-03-02T10:02:00',56.02, 47.26),
    ( 1, '2019-03-02T11:11:00',51.67, 40.42),
    ( 2, '2019-03-01T12:27:00',55.13, 42.97),
    ( 2, '2019-03-01T16:10:00',53.36, 43.96),
    ( 2, '2019-03-02T00:03:00',53.06, 40.17),
    ( 2, '2019-03-02T06:41:00',59.11, 49.68),
    ( 2, '2019-03-02T11:08:00',58.28, 40.78),
    ( 2, '2019-03-02T12:52:00',55.61, 43.31),
    ( 3, '2019-03-01T10:03:00',58.51, 40.40),
    ( 3, '2019-03-01T17:58:00',59.29, 45.87),
    ( 3, '2019-03-01T20:17:00',55.24, 48.44),
    ( 3, '2019-03-02T03:59:00',59.25, 48.37),
    ( 3, '2019-03-02T07:45:00',59.01, 48.81),
    ( 3, '2019-03-02T12:26:00',58.50, 44.37),
    ( 4, '2019-03-01T10:51:00',53.60, 42.50),
    ( 4, '2019-03-01T17:52:00',57.64, 41.97),
    ( 4, '2019-03-01T19:51:00',51.55, 45.07),
    ( 4, '2019-03-01T22:42:00',52.01, 48.96),
    ( 4, '2019-03-02T05:56:00',58.21, 40.79),
    ( 4, '2019-03-02T10:46:00',56.09, 47.75)
 ]
geo_df = pd.DataFrame ( geo_df , columns = ['user_id','time','lat','lon'])
geo_df.sort_values(['time'],inplace=True)
geo_df

,user_id,time,lat,lon
12,3,2019-03-01T10:03:00,58.51,40.40
0,1,2019-03-01T10:14:00,54.88,42.93
18,4,2019-03-01T10:51:00,53.60,42.50
6,2,2019-03-01T12:27:00,55.13,42.97
1,1,2019-03-01T15:20:00,57.89,48.04
7,2,2019-03-01T16:10:00,53.36,43.96
19,4,2019-03-01T17:52:00,57.64,41.97
13,3,2019-03-01T17:58:00,59.29,45.87
2,1,2019-03-01T18:46:00,51.61,44.52
20,4,2019-03-01T19:51:00,51.55,45.07


Визиты пользователей. Данные о сохраняются в БД в момент входа в придожение и т.к. они не привязаны  к получению данных о положении пользователя, их обычно меньше, чем данных о координатах.

In [12]:
vis_df = [
    ( 1, '2019-03-01T18:52:00'),
    ( 1, '2019-03-01T15:35:00'),
    ( 2, '2019-03-02T11:18:00'),
    ( 2, '2019-03-01T12:43:00'),
    ( 3, '2019-03-01T10:05:00'),
    ( 3, '2019-03-01T20:32:00'),
    ( 4, '2019-03-01T22:58:00'),
    ( 4, '2019-03-01T18:05:00')
]
vis_df = pd.DataFrame (vis_df,columns=['user_id','time'])
vis_df.sort_values(['time'],inplace=True)
vis_df

,user_id,time
4,3,2019-03-01T10:05:00
3,2,2019-03-01T12:43:00
1,1,2019-03-01T15:35:00
7,4,2019-03-01T18:05:00
0,1,2019-03-01T18:52:00
5,3,2019-03-01T20:32:00
6,4,2019-03-01T22:58:00
2,2,2019-03-02T11:18:00


Покупки пользователей. Данные сохраняются  в момент покупки и их может быть больше или меньше, чем визитов.

In [17]:
buy_df = [
    ( 1, '2019-03-01T19:29:00',68.38),
    ( 2, '2019-03-01T13:15:00',32.53),
    ( 3, '2019-03-01T10:21:00',23.27),
    ( 4, '2019-03-01T23:02:00',39.40)
]
buy_df = pd.DataFrame(buy_df,columns=['user_id','time','cost'])
buy_df

,user_id,time,cost
0,1,2019-03-01T19:29:00,68.38
1,2,2019-03-01T13:15:00,32.53
2,3,2019-03-01T10:21:00,23.27
3,4,2019-03-01T23:02:00,39.40


## Подход к  решению
Данные о визитах и гео-координат будем сопоставлять через **user_id** и **time**. Но т.к. прямое сопоставление через time невозможно т.к. крайне маловероятно, что два события проихойдут одновременно, то неоходимо продумать механизм как сопоставлять по времени основываясь не на точных сопоставлениях.

Для решения задачи необходимо сопоставить каждый визит с какой-то геометкой. Я считаю, что необходимо сопоставлять с ближайшей гео-меткой в прошлом. Для этого для каждого визита выберем все метки для данного пользователя в прошлом, и поскольку они будут отсортированы по времени, просто сохраним индекс последней существующей геометки в поле **geo_id** для каждого визита. В последствии произвести слияние двух наборов данных : визиты и гео-метки по полю geo_id.

*Можно модифицировать код так чтобы брать просто ближайшую гео-метку а не только из прошлого*

In [13]:
vis_df['geo_id'] = pd.Series([]) # просто добавляем новый столбец к фрейму

for vis in vis_df.itertuples():
    uid = vis.user_id
    tme = vis.time
    
    # это создаёт фрейм с гео-метакми в прошлом для данного пользователя
    tmp_df = geo_df[geo_df['user_id']==uid].copy()
    tmp_df = tmp_df[tmp_df['time'] < tme]
    
    # если там вообще есть хоть какие-то гео-метки,
    # то последнюю сохраняем в набор визитов
    if not tmp_df.empty:
        for geo in tmp_df[-1:].itertuples():
            vis_df['geo_id'].update(pd.Series([geo.Index],index=[vis.Index]))

# слияние наборов данных по полю geo_id
vis_df = vis_df.merge(geo_df,how='left',left_on='geo_id',right_index=True,suffixes=('','_geo')).drop(['user_id_geo'],axis=1)
vis_df.head()

,user_id,time,geo_id,time_geo,lat,lon
4,3,2019-03-01T10:05:00,12.0,2019-03-01T10:03:00,58.51,40.40
3,2,2019-03-01T12:43:00,6.0,2019-03-01T12:27:00,55.13,42.97
1,1,2019-03-01T15:35:00,1.0,2019-03-01T15:20:00,57.89,48.04
7,4,2019-03-01T18:05:00,19.0,2019-03-01T17:52:00,57.64,41.97
0,1,2019-03-01T18:52:00,2.0,2019-03-01T18:46:00,51.61,44.52
5,3,2019-03-01T20:32:00,14.0,2019-03-01T20:17:00,55.24,48.44
6,4,2019-03-01T22:58:00,21.0,2019-03-01T22:42:00,52.01,48.96
2,2,2019-03-02T11:18:00,10.0,2019-03-02T11:08:00,58.28,40.78


Для покупок подход аналогичен, но не совсем. 

Сопоставление набора покупок проводим с набором визитов, и гео-метку будем брать, не ту которая последняя, а та которая в визите. *Тут конечно определяет бизнес-логика с каким набором сопоставлять, но я предположил, что лучше с визитами.*

In [18]:
buy_df['vis_id'] = pd.Series([]) # просто добавляем новый столбец к фрейму

for buy in buy_df.itertuples():
    uid = buy.user_id
    tme = buy.time

    tmp_df = vis_df[vis_df['user_id']==uid].copy()
    tmp_df = tmp_df[tmp_df['time'] < tme]

    if not tmp_df.empty:
        for vis in tmp_df[-1:].itertuples():    
            buy_df['vis_id'].update(pd.Series([vis.Index],index=[buy.Index]))

buy_df = buy_df.merge(vis_df,how='left',left_on='vis_id',right_index=True,suffixes=('','_vis')).drop(['user_id_vis'],axis=1)
buy_df.head()

,user_id,time,cost,vis_id,time_vis,geo_id,time_geo,lat,lon
0,1,2019-03-01T19:29:00,68.38,0.0,2019-03-01T18:52:00,2.0,2019-03-01T18:46:00,51.61,44.52
1,2,2019-03-01T13:15:00,32.53,3.0,2019-03-01T12:43:00,6.0,2019-03-01T12:27:00,55.13,42.97
2,3,2019-03-01T10:21:00,23.27,4.0,2019-03-01T10:05:00,12.0,2019-03-01T10:03:00,58.51,40.40
3,4,2019-03-01T23:02:00,39.40,6.0,2019-03-01T22:58:00,21.0,2019-03-01T22:42:00,52.01,48.96


ПРОФИТ !!